### Similarity function

In [2]:
import pandas as pd
from scipy import spatial

In [11]:
cbb21 = pd.read_csv('CSV_Files/cbb21_top_68_clean.csv', index_col = 0)
cbb_team_avg = pd.read_csv('CSV_Files/avg_by_round.csv', index_col = 0)
cbb_teams = pd.read_csv('CSV_Files/cbb_teams.csv', index_col = 0)

In [4]:
cbb21

,TEAM,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,Michigan,118.1,91.1,0.9521,54.9,44.9,16.3,15.1,29.4,24.8,28.9,24.5,53.3,42.3,38.7,33.5,66.9,7.2,1.0
1,Baylor,123.2,94.5,0.9548,57.5,49.1,17.6,24.6,37.5,30.9,27.0,31.7,54.1,48.1,41.8,34.0,68.8,6.6,1.0
2,Illinois,117.7,90.4,0.9539,55.6,46.6,18.2,16.1,33.0,22.2,39.2,30.5,55.3,45.4,37.6,32.7,70.7,8.9,1.0
3,Gonzaga,125.4,89.8,0.9791,61.0,47.5,16.1,20.3,30.4,23.4,36.7,25.9,64.0,46.8,36.5,32.5,74.6,8.5,1.0
4,Iowa,123.5,95.7,0.9491,54.6,48.3,13.3,16.3,30.7,28.6,32.0,22.6,52.4,45.8,38.6,34.8,70.0,5.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Mount St. Mary's,95.7,100.6,0.3593,48.7,45.5,20.1,16.3,31.7,24.6,29.7,24.6,47.7,45.6,33.7,30.1,62.2,-7.2,16.0
64,Hartford,97.3,99.6,0.4345,50.4,47.6,19.0,20.4,24.8,28.6,28.6,25.0,51.3,50.7,32.7,28.7,66.9,-4.9,16.0
65,Norfolk St.,99.4,104.1,0.3698,49.7,47.4,17.8,20.7,28.8,28.7,38.6,42.0,46.1,47.2,37.4,31.7,67.9,-5.0,16.0
66,Texas Southern,97.3,103.5,0.3307,48.3,46.5,21.4,19.2,32.9,25.7,38.9,33.8,51.0,45.2,27.7,32.4,71.8,-4.5,16.0


In [5]:
# checking the teams most correlated similarity to a seed

def full_similarity(df1, df2):
    final_results = []
    for x in df1.TEAM:
        max_similarity = 0
        current = []
        
        for y in df2.POSTSEASON:
        
            similarity = 1/(spatial.distance.euclidean(df1[df1.TEAM == x].drop(columns = ['TEAM', 'SEED', 'BARTHAG']), 
                                         df2[df2.POSTSEASON == y].drop(columns = ['POSTSEASON', 'BARTHAG'])))
            if similarity > max_similarity:
                max_similarity = similarity
                current_team = x
                team_seed = df1[df1.TEAM == x]['SEED'].unique()[0]
                current_postseason_rank = y
        current.append(current_team)
        current.append(int(team_seed))
        current.append(max_similarity)
        current.append(current_postseason_rank)
        final_results.append(current)
    
    return final_results
        


In [12]:
Teams_by_avg_postseason = pd.DataFrame(full_similarity(cbb21, cbb_team_avg), 
                            columns = ['Team', 'Team_Seed', 'Similarity_Score', 'Postseason_Finish'])


In [13]:
# checking each teams similarity to all the different seeds

def full_similarity2(df1, df2):
    final_results = []
    
    for x in df1.TEAM:
        max_similarity = 0
        
        for y in df2.POSTSEASON:
            similarity = 1/(spatial.distance.euclidean(df1[df1.TEAM == x].drop(columns = ['TEAM', 'SEED', 'BARTHAG']), 
                                         df2[df2.POSTSEASON == y].drop(columns = ['POSTSEASON', 'BARTHAG'])))
            if similarity > max_similarity:
                max_similarity = similarity
                current_team = x
                current_postseason_rank = y
            final_results.append(x + ', ' + str(similarity) + ', ' + str(y))
    
    return final_results

In [14]:
full_similarity2(cbb21, cbb_team_avg)

['Michigan, 0.04601141080349276, 1',
 'Michigan, 0.05919638094229621, 2',
 'Michigan, 0.06957271482498859, 3',
 'Michigan, 0.07168206249120586, 4',
 'Michigan, 0.07260177485779545, 5',
 'Michigan, 0.09031491219149301, 6',
 'Michigan, 0.0808325405058656, 7',
 'Baylor, 0.04208584295894996, 1',
 'Baylor, 0.052805475439480005, 2',
 'Baylor, 0.06000168657621287, 3',
 'Baylor, 0.06293668986042718, 4',
 'Baylor, 0.06251959651540495, 5',
 'Baylor, 0.061581221083492406, 6',
 'Baylor, 0.0727941822884859, 7',
 'Illinois, 0.05194379258979546, 1',
 'Illinois, 0.07115017932389042, 2',
 'Illinois, 0.08735742048247966, 3',
 'Illinois, 0.10438545931651692, 4',
 'Illinois, 0.08462140737339212, 5',
 'Illinois, 0.10660715732047107, 6',
 'Illinois, 0.09269506962200827, 7',
 'Gonzaga, 0.034042751029916794, 1',
 'Gonzaga, 0.040638548170635426, 2',
 'Gonzaga, 0.04581389176768351, 3',
 'Gonzaga, 0.051827949145015186, 4',
 'Gonzaga, 0.04588661920659803, 5',
 'Gonzaga, 0.05702564868935496, 6',
 'Gonzaga, 0.05831

### Expanded for comparison to all the s16 teams

In [17]:
cbb_teams_indexed = cbb_teams.reset_index()

In [22]:
# checking the teams most correlated similarity to a team historically

def full_similarity_all(df1, df2):
    final_results = []
    
    for x in df1.TEAM:
        current = []
        max_similarity = 0
        
        for y in df2.index:
            similarity = 1/(spatial.distance.euclidean(df1[df1.TEAM == x].drop(columns = ['TEAM', 'SEED', 'BARTHAG']), 
                                         df2[df2.index == y].drop(columns = ['index', 'TEAM', 'SEED', 'POSTSEASON', 'BARTHAG', 'YEAR'])))
            
            if similarity > max_similarity:
                max_similarity = similarity
                current_team = x
                team_seed = df1[df1.TEAM == x]['SEED'].unique()[0]
                current_postseason_rank = df2[df2.index == y]['POSTSEASON'].unique()[0]
                current_compared_team = df2[df2.index == y]['TEAM'].unique()[0]
                current_compared_team_year = df2[df2.index == y]['YEAR'].unique()[0]
        current.append(current_team)
        current.append(int(team_seed))
        current.append(max_similarity)
        current.append(current_compared_team)
        current.append(current_compared_team_year)
        current.append(current_postseason_rank)
        final_results.append(current)
        
        
    return final_results
        

In [24]:
Team_comparison = pd.DataFrame(full_similarity_all(cbb21, cbb_teams_indexed), 
                              columns = ['Active_Team', 'Team_Seed', 'Similarity_Score', 
                                         'Best_Comparison_Team', 'Best_Comparison_Team_Season','Postseason_Finish'])


In [25]:
Team_comparison

,Active_Team,Team_Seed,Similarity_Score,Best_Comparison_Team,Best_Comparison_Team_Season,Postseason_Finish
0,Michigan,1,0.110885,Purdue,2017,3
1,Baylor,1,0.092159,Purdue,2019,4
2,Illinois,1,0.106770,Kentucky,2019,4
3,Gonzaga,1,0.120789,Gonzaga,2019,4
4,Iowa,2,0.123504,Purdue,2018,3
...,...,...,...,...,...,...
63,Mount St. Mary's,16,0.075190,Radford,2018,1
64,Hartford,16,0.075399,UMBC,2018,2
65,Norfolk St.,16,0.106119,Coastal Carolina,2014,1
66,Texas Southern,16,0.109415,Hampton,2016,1


In [26]:
Teams_by_avg_postseason.to_csv('CSV_Files/Teams_by_avg_postseason.csv')
Team_comparison.to_csv('CSV_Files/Team_comparison.csv')